#Yellow Pages Canada

In [5]:
import json, requests, urllib, re, pprint

In [1]:
YP_Local_AD_Network_api_key_request_url = 'https://publisher.yp.com/register'
yellow_api_search_key_request_url = 'http://developer.yellowapi.com/member/register'

##YellowAPI
http://www.yellowapi.com/docs/  
http://www.yellowapi.com/docs/places/  
http://www.yellowapi.com/docs/deals/  

In [3]:
sandbox_api = ''
production_api = ''
sandbox_root = 'http://api.sandbox.yellowapi.com'
production_root = 'http://api.yellowapi.com'

###Places API Methods
**FindBusiness** - This request returns local Canadian businesses that are most relevant to the search criteria.  
**GetBusinessDetails** - Get business details.  Requires a FindBusiness call first. Information returned on this first call provides the necessary information for the GetBusinessDetails call.  
**FindDealer** - When applicable, this request returns a list of all dealers/franchises/branches for that business. A business with several locations may have a “parent” listing (primary) and one or more “children” listings.  
**GetTypeAhead** - USed for an autocomplete form  

In [246]:
class sandbox():
    '''
    Yellow Pages Canada Sandbox API implementation
    create an instance of sandbox
    
    instance = sandbox()
    
    the search methods have parameters.  They are available for reference through the instance $Method_paramaters
    
    instance.Business_parameters
    
    build a dict prom selected paramaters for your method and pass that to the $Method_search function.
    
    params = { 'what':'Boots' }
    instance.Business_Search( params )
    '''
    
    global sandbox_api
    global sandbox_root
    global Business_Search_parameters
    global Business_Details_Search_parameters
    global FindDealer_Search_paramaters
    global GetTypeAhead_Search_parameters
    sandbox_api = ''
    sandbox_root = 'http://api.sandbox.yellowapi.com'
    
    Business_Search_parameters = {
        'pg':{ 'description':'Requested page', 'example':'integer > 0, max 50 defailt 1', 'required':'N'},
        'what':{ 'description':'search term or keyword or telephone number', 'example':'any text string, utf-8', 'required':'Y'},
        'lang':{ 'description':'language for returned content', 'example':'en, fr', 'required':'N'},
        'where':{ 'description':'This may be a location name or a specific coordinate specification in the format: cZ{longitude},{latitude}.', 'example':'Toronto , cZ-73.61995,45.49981, UTF-8', 'required':'Y' },
        'pgLen':{ 'descrition':'Number of results', 'example':'integer > 0, Max 100, Default 40', 'required':'N'},
        'dist':{ 'description':'Maximum distance to return results within (when latitude/longitude is specified) in kilometres', 'example':'Positive decimal value', 'required':'N'},
        'fmt':{ 'desription':'The format of output', 'example':'JSON or XML', 'required':'Y'},
        'sflag':{ 'description':'Flags to modify or filter the search result. These may be combined as logical “AND” filters.  To combine more than one filter, use the “dash” to separate each value.', 'example':'bn – {what} search term in business name, fto – results has photo, vdo – results has video', 'required':'N'},
        'apikey':{ 'description':'an API key for the Places API', 'example':'24 characters alphanumeric string', 'required':'Y'},
        'UID':{ 'description':'A string of characters which uniquely identifies a user of the application. (ie. IP address, session ID, or hash of the phone ID)', 'example':'IP address, session ID, hash of the phone ID, etc.', 'required':'Y' }
    }
    Business_Details_Search_parameters = {
        'prov':{ 'description':'Normalized name of Province.  Canada is acceptable', 'example':'text-string', 'required':'Y' },
        'city':{ 'description':'The city location', 'example':'Text string', 'required':'N' },
        'bus-name':{ 'description':'Business Name normalized', 'example':'text-string', 'required':'Y' },
        'listingId':{ 'description':'The unique listing id identifying the business', 'example':'from getbusiness and getdealer', 'required':'Y' },
        'lang':{ 'description':'Suggestion Language', 'example':'en, fr', 'required':'N' },
        'fmt':{ 'description':'The format of the Output', 'example':'JSON or XML', 'required':'Y' },
        'apikey':{ 'description':'API key', 'example':'24 character alphanumeric', 'required':'Y' },
        'UID':{ 'description':'A string of characers which uniquely identifies a user of the application', 'example':'phone-number or proper name', 'required':'Y' },
    }
    FindDealer_Search_paramaters = {
        'pid':{ 'description':'The listingId of a parent business. A parent business can be identified by the ‘isParent’ flag in the Listing object returned by FindBusinesses.', 'example':'Valid parent listing id', 'required':'Y' },
        'pg':{ 'description':'The requested page', 'example':'Number > 0 <=50 default 1', 'required':'N' },
        'pgLen':{ 'description':'Results to return per page', 'example':'Number > 0 and <= 100 default 40', 'required':'N' },
        'lang':{ 'description':'Suggestion language', 'example':'en, fr', 'required':'N' },
        'fmt':{ 'description':'THe format of the ouput', 'example':'JSON or XML', 'required':'Y' },
        'apikey':{ 'description':'API key', 'example':'24 character alphanumeric', 'required':'Y' },
        'UID':{ 'description':'A string of characers which uniquely identifies a user of the application', 'example':'phone-number or proper name', 'required':'Y' }, 
    }
    
    GetTypeAhead_Search_parameters = {
        'text':{ 'description':'Characters typed by user', 'example':'Any Character sequence', 'required':'Y' },
        'lang':{ 'description':'Suggestion language', 'example':'en, fr', 'required':'N' },
        'field':{ 'description':'which field to provide suggestion', 'example':'WHAT, WHERE', 'required':'Y' },
        'apikey':{ 'description':'the api key', 'example':'24 digit alpha numeric code', 'required':'Y' },
        'fmt':{ 'description':'THe format of the ouput', 'example':'JSON or XML', 'required':'Y' },
        'UID':{ 'description':'A unique identifier', 'example':'Phone number, proper name', 'required':'Y' },
    }
    
    def __init__(self):
        self.sandbox_api = sandbox_api
        self.sandbox_root = sandbox_root
        self.Business_Search_parameters = Business_Search_parameters
        self.Business_Details_Search_parameters = Business_Details_Search_parameters
        self.FindDealer_Search_paramaters = FindDealer_Search_paramaters
        self.GetTypeAhead_Search_parameters = GetTypeAhead_Search_parameters
        
    def FindBusiness_Search_Required_Parameters(self):
        return [p for p in Business_Search_parameters if Business_Search_parameters[p]['required'] == 'Y']
    
    def GetBusinessDetails_Search_Required_Parameters(self):
        return [p for p in Business_Details_Search_parameters if Business_Details_Search_parameters[p]['required'] == 'Y']
    
    def FindDealer_Search_Required_Paramaters(self):
        return [p for p in FindDealer_Search_paramaters if FindDealer_Search_paramaters[p]['required'] == 'Y']
    
    def GetTypeAhead_Search_Required_Parameters(self):
        return [p for p in GetTypeAhead_Search_parameters if GetTypeAhead_Search_parameters[p]['required'] == 'Y']
    
    def FindBusiness_Search(self, parameters_dict):
        '''
        build a dict prom selected paramaters for your method and pass that to the $Method_search function.
        
        instance = sandbox()
        params = { 'what':'Boots' }
        instance.Business_Search( params )
        
        All Required Paramaters have to be passed in
        [p for p in instance.Business_parameters if p.values()[0]['required' == 'Y']
        
        specify JSON output 'fmt':'JSON'
        '''
        form = params
        query = urllib.urlencode(form)
        #print (sandbox_root + '/FindBusiness/' + "?" + query)
        YellowAPI_request = urllib.urlopen(sandbox_root + '/FindBusiness/' + "?" + query)
        x = json.loads(YellowAPI_request.read())
        pprint.pprint(x)
    
    def GetBusinessDetails_Search(self, parameters_dict):
        '''
        pass in the business id from FindBusiness() in the paramaters
        '''
        form = params
        query = urllib.urlencode(form)
        #print (sandbox_root + '/GetBusinessDetails/' + "?" + query)
        YellowAPI_request = urllib.urlopen(sandbox_root + '/GetBusinessDetails/' + "?" + query)
        x = json.loads(YellowAPI_request.read())
        pprint.pprint(x)
    
    def FindDealer_Search(self, paramaters_dict):
        '''
        pass in the parentId in params.
        '''
        form = params
        query = urllib.urlencode(form)
        #print (sandbox_root + '/FindDealer/' + "?" + query)
        YellowAPI_request = urllib.urlopen(sandbox_root + '/FindDealer/' + "?" + query)
        x = json.loads(YellowAPI_request.read())
        pprint.pprint(x)
    
    def GetTypeAhead_Search(self, parameters_dict):
        '''
        pass in required paramaters in dict form.
        
        This is for autocomplete for a web form. field is WHAT or WHERE
        '''
        form = params
        query = urllib.urlencode(form)
        #print (sandbox_root + '/GetTypeAhead/' + "?" + query)
        YellowAPI_request = urllib.urlopen(sandbox_root + '/GetTypeAhead/' + "?" + query)
        x = json.loads(YellowAPI_request.read())
        pprint.pprint(x)

In [247]:
instance = sandbox()
dir(instance)

['Business_Details_Search_parameters',
 'Business_Search_parameters',
 'FindBusiness_Search',
 'FindBusiness_Search_Required_Parameters',
 'FindDealer_Search',
 'FindDealer_Search_Required_Paramaters',
 'FindDealer_Search_paramaters',
 'GetBusinessDetails_Search',
 'GetBusinessDetails_Search_Required_Parameters',
 'GetTypeAhead_Search',
 'GetTypeAhead_Search_Required_Parameters',
 'GetTypeAhead_Search_parameters',
 '__doc__',
 '__init__',
 '__module__',
 'sandbox_api',
 'sandbox_root']

In [228]:
instance.FindBusiness_Search_Required_Parameters()

['what', 'apikey', 'fmt', 'where', 'UID']

In [179]:
params = { 'what':'Coffee', 'apikey':sandbox_api, 'where':'Toronto', 'UID':'crispybacon', 'fmt':'JSON' }
instance.FindBusiness_Search( params )

{u'listings': [{u'address': {u'city': u'Toronto',
                             u'pcode': u'M6J 1Y3',
                             u'prov': u'ON',
                             u'street': u'1375 Dundas St W'},
                u'content': {u'DspAd': {u'avail': False, u'inMkt': False},
                             u'Logo': {u'avail': False, u'inMkt': False},
                             u'Photo': {u'avail': False, u'inMkt': False},
                             u'Profile': {u'avail': True, u'inMkt': True},
                             u'Url': {u'avail': False, u'inMkt': False},
                             u'Video': {u'avail': False, u'inMkt': False}},
                u'distance': u'3.8',
                u'geoCode': {u'latitude': u'43.649541',
                             u'longitude': u'-79.427902'},
                u'id': u'100744375',
                u'isParent': False,
                u'name': u'Solita',
                u'parentId': u''},
               {u'address': {u'city': u'Toronto'

In [236]:
instance.GetBusinessDetails_Search_Required_Parameters()

['apikey', 'UID', 'listingId', 'prov', 'fmt', 'bus-name']

In [237]:
params = { 'apikey':sandbox_api, 'UID':'Jesse Bacon', 'listingId':'100744375', 'prov':'Canada', 'fmt':'JSON', 'bus-name':'Solita'}
instance.GetBusinessDetails_Search( params )

{u'address': {u'city': u'Toronto',
              u'pcode': u'M6J 1Y3',
              u'prov': u'ON',
              u'street': u'1375 Dundas St W'},
 u'categories': [{u'isSensitive': False, u'name': u'Cafes Terraces'},
                 {u'isSensitive': False, u'name': u'Gourmet Food Shops'}],
 u'geoCode': {u'latitude': u'43.649541', u'longitude': u'-79.427902'},
 u'id': u'100744375',
 u'logos': {u'EN': u'http://cdn.ci2.yp.ca/l/1596/84/15968499aa_l.gif',
            u'FR': u'http://cdn.ci3.yp.ca/t/1596/85/15968515aa_t.gif'},
 u'merchantUrl': u'http://www.yellowpages.ca/bus/Ontario/Toronto/Solita/100744375.html',
 u'name': u'Solita',
 u'phones': [{u'dispNum': u'416-519-5534',
              u'npa': u'416',
              u'num': u'5534',
              u'nxx': u'519',
              u'type': u'primary'}],
 u'products': {u'dispAd': [],
               u'photos': [],
               u'profiles': [{u'headerImage': None,
                              u'lang': u'FR',
                              u'

In [212]:
instance.FindDealer_Search_Required_paramaters()

['apikey', 'UID', 'fmt', 'pid']

In [224]:
params = { 'apikey':sandbox_api, 'UID':'Jesse Bacon', 'listingId':'6418182', 'prov':'Canada', 'fmt':'JSON', 'bus-name':'The Co-operators'}
instance.GetBusinessDetails_Search(params)

{u'address': {u'city': u'', u'pcode': u'', u'prov': u'', u'street': u''},
 u'categories': [{u'isSensitive': False, u'name': u'General Insurance'},
                 {u'isSensitive': False,
                  u'name': u'Insurance Agents & Brokers'}],
 u'id': u'6418182',
 u'logos': {u'EN': u'http://cdn.ci2.yp.ca/t/1577/09/15770919aa_t.gif',
            u'FR': u'http://cdn.ci1.yp.ca/t/1582/91/15829183aa_t.gif'},
 u'merchantUrl': u'http://www.yellowpages.ca/bus/Canada/The-Co-operators/6418182.html',
 u'name': u'The Co-operators',
 u'phones': [],
 u'products': {u'dispAd': [{u'thmbUrl': u'http://cdn.ci2.yp.ca/l/1548/53/15485359aa_l.gif',
                            u'url': u'http://cdn.ci3.yp.ca/f/1548/53/15485359aa_f.jpg'}],
               u'photos': [],
               u'profiles': [{u'keywords': {u'LangSpk': [u'Anglais'],
                                            u'OpenHrs': [u'Lundi 8:30 - 17:00',
                                                         u'Mardi 8:30 - 17:00',
            

In [248]:
instance.GetTypeAhead_Search_Required_Parameters()

['apikey', 'UID', 'text', 'fmt', 'field']

In [245]:
params = { 'text':'Mart', 'apikey':sandbox_api, 'UID':'Jesse Bacon', 'field':'WHAT', 'fmt':'JSON' }
instance.GetTypeAhead_Search( params )

{u'suggestedValues': [{u'alt': u'martial arts lessons & schools',
                       u'code': u'',
                       u'type': u's',
                       u'value': u'martial arts lessons & schools'},
                      {u'alt': u'martial arts',
                       u'code': u'',
                       u'type': u's',
                       u'value': u'martial arts'},
                      {u'alt': u'Martin',
                       u'code': u'',
                       u'type': u's',
                       u'value': u'Martin'},
                      {u'alt': u'martial arts supplies',
                       u'code': u'',
                       u'type': u's',
                       u'value': u'martial arts supplies'},
                      {u'alt': u'Martel',
                       u'code': u'',
                       u'type': u's',
                       u'value': u'Martel'},
                      {u'alt': u'martial arts & self-defense instruction',
                       u'